# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1 [10 баллов] до 23:59 17.02.2018

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе с gismeteo.ru
2. Генерация описания прогноза
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



### 1. Сбор данных [3 балла]

Выберите произвольным образом город в России и найдите прогноз погоды в нем на ближайшие 10 дней на сайте gismeteo.ru.

Пример: прогноз на 10 ближайших дней в Москве – https://www.gismeteo.ru/weather-moscow-4368/10-days/

Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней, начиная со дня, когда вы начали делать домашнее задание.  

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальна температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.  Пример итоговой таблицы вы найдете в следующей части задания. 

In [17]:
from bs4 import BeautifulSoup
import requests as req
import user_agent as ua
import datetime
import pandas as pd
import pymorphy2
from itertools import combinations

In [18]:
url = 'https://www.gismeteo.ru/weather-moscow-4368/10-days/'

# Gismeteo иногда не дает доступ с питоновским user-agent'ом, установим человеческй
user_agent = ua.generate_user_agent(os='linux')


headers = req.utils.default_headers()
headers.update({'User-Agent': user_agent})

res = req.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'lxml')

In [19]:
w_days = list(map(lambda s: s.text.strip().lower(), soup.findAll('div', 'w_date__day')[:10]))
dates = list(map(lambda s: str(s.text.strip().split()[0])+'.02', soup.findAll('span', 'w_date__date')[:10]))
min_temps = list(map(lambda s: s.text.strip() if ord(s.text.strip()[0]) != 8722 else '-'+s.text.strip()[1:],
                     soup.findAll('div', 'mint')[:10]))
max_temps = list(map(lambda s: s.text.strip() if ord(s.text.strip()[0]) != 8722 else '-'+s.text.strip()[1:],
                     soup.findAll('div', 'maxt')[:10]))
w_speed = list(map(lambda s: s.text.strip(), soup.findAll('div', 'w_wind__warning')[:10]))
prec = list(map(lambda s: s.text.strip().replace(',', '.'), soup.findAll('div', 'w_precipitation__value')[:10]))

info = {}
for i in range(10):
    date_str = dates[i]+' ('+w_days[i]+')'
    info.update({date_str: [min_temps[i], max_temps[i], w_speed[i], prec[i]]})

In [20]:
df = pd.DataFrame(data=info, index=['минимальная температура', 
                                   'максимальная температура', 
                                   'скорость ветра',
                                   'уровень осадков'])

In [21]:
df.head()

,18.02 (вс),19.02 (пн),20.02 (вт),21.02 (ср),22.02 (чт),23.02 (пт),24.02 (сб),25.02 (вс),26.02 (пн),27.02 (вт)
минимальная температура,-6,-6,-9,-14,-17,-16,-18,-20,-23,-23
максимальная температура,-3,-3,-6,-10,-11,-11,-12,-16,-17,-15
скорость ветра,4,6,8,8,7,5,8,9,9,9
уровень осадков,1.85,1.5,0.8,0.1,0.1,0.2,0,0,0,0


In [22]:
df.to_csv('data_from_gismeteo.csv')

---

### 2. Генератор описания прогноза погоды [4 балла]

Если у вас не получилось извлечь прогноз погоды в предыдущей части задания, воспользуйтесь таблицей ниже.
В ней приведен прогноз четырех показателей на ближайшие 10 дней в Москве – минимальная и максимальная температура, скорость ветра и уровень осадков. 

|                | 02.02 (пт) | 03.02 (сб) | 04.02 (вс)| 05.02 (пн) | 06.02 (вт) | 07.02 (пн) | 08.02 (ср) | 09.02 (ср) | 10.02 (сб) | 11.02 (вс)
|----------------|-------|-------|-------|-------|-------|-------|-------|
| минимальная температура    | -9    | -1    | -8    | -13    | -12    | -15    | -21    | -14 |-8 |-8
| максимальная температура    | -1    | +1    | -2    | -9   | -11    | -12    | -16    |-5    |-6    |-5|
| скорость ветра | 10    | 13    | 15    | 15   |11    | 6    | 7 | 9 | 8 |12
| уровень осадков         | 1.35  | 8.6  | 15.5  | 6.6   | 2.7   | 2.1   | 0   | 3.2   |0.8  | 0.4

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 единицы в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 единиц за 7 дней*

Вместо НазваниеГорода подставьте название выбранного вами города, используя фунцкии для согласования существительных с предлогами. 

Некоторые вспомогательные функции, которые вам понадобятся: согласование существительного с числительным и приведение существительного к нужному падежу: 

In [23]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
word = 'яблоко'
parsed_word = morph.parse(word)[0]
print(1, parsed_word.make_agree_with_number(1).word) # согласование слова с числительным 1
print(2, parsed_word.make_agree_with_number(2).word) # согласование слова с числительным 2
print(3, parsed_word.make_agree_with_number(5).word) # согласование слова с числительным 5

1 яблоко
2 яблока
3 яблок


In [24]:
print(parsed_word.inflect({'gent'}).word) # слово в родительном падеже

яблока


Ниже приведен пример решения аналогичной задачи генерации текстов: генератор отчета о том, сколько и каких фруктов съел Вася.

In [25]:
def eating(n, fruit):
    parsed_word = morph.parse(fruit)[0]
    s = ' '.join(['Вася съел', str(n) ,parsed_word.make_agree_with_number(n).word, '.'])
    return s

In [26]:
eating(5, 'яблоко')

'Вася съел 5 яблок .'

In [27]:
eating(4, 'груша')

'Вася съел 4 груши .'

Пример более сложного генератора: фрукты могут есть не только мальчики, но и девочки, то есть, нужно не только согласовать числительное с существительным, но и поставить глагол в нужную форму.

In [28]:
def eating(name, n, fruit):
    name_tag = morph.parse(name)[0].tag
    if 'masc' in name_tag:
        gender = 'masc'
    if 'femn' in name_tag:
        gender = 'femn'
    verb = morph.parse("съесть")[0].inflect({'perf', gender,'sing','past','indc'}).word
    parsed_word = morph.parse(fruit)[0]
    s = ' '.join([name, verb, str(n) ,parsed_word.make_agree_with_number(n).word, '.'])
    return s

In [29]:
eating('Маша', 3, 'вишня')

'Маша съела 3 вишни .'

---

In [332]:
# utils
def parsing_word(word):
    morph = pymorphy2.MorphAnalyzer()
    return morph.parse(word)[0]

def get_in_form(day, from_upper=False):
    return ('В' if from_upper else 'в') if day.lower() != 'вторник' else ('Во' if from_upper else 'во')

def get_with_form(day, from_upper=False):
    return ('С' if from_upper else 'с') if day.lower() not in ['вторник', 'среда'] else\
                                                    ('Со' if from_upper else 'cо')

def get_full_w_day(short_day):
    w_days_map = {'пн': 'понедельник',
                  'вт': 'вторник',
                  'ср': 'среда',
                  'чт': 'четверг',
                  'пт': 'пятница',
                  'сб': 'суббота',
                  'вс': 'воскресенье'}
    return w_days_map[short_day]

def make_agree_with_number(word, num):
    return parsing_word(word).make_agree_with_number(num).word

def inflect(word, form):
    return parsing_word(word).inflect(form).word

In [30]:
# В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
def make_first_sentence(day1, day2, diff_temps): 
    day1, day2 = get_full_w_day(day1), get_full_w_day(day2)
    s = ' '.join([get_in_form(day2, from_upper=True), inflect(day2, {'NOUN', 'accs'}), 
                  'в', inflect('Москва', {'NOUN', 'datv'}).capitalize(),
                  'потелеет' if int(diff_temps)>0 else 'похолодает', 'на', str(abs(diff_temps)), 
                  make_agree_with_number('градус', abs(diff_temps)), 'по сравнению', get_with_form(day1), 
                  inflect(day1, {'NOUN', 'ablt'})])
    return s

In [31]:
make_first_sentence('ср', 'вт', -4)

'Во вторник в Москве похолодает на 4 градуса по сравнению cо средой'

In [32]:
# Скорость ветра изменится на X единиц в день1 по сравнению с день2
def make_sec_sentence(day1, day2, diff):
    day1, day2 = get_full_w_day(day1), get_full_w_day(day2)
    return ' '.join(['Скорость ветра изменится на', str(abs(diff)), make_agree_with_number('единица', abs(diff)), 
                     get_in_form(day1), inflect(day1, {'NOUN', 'accs'}), 'по сравнению', 
                     get_with_form(day2), inflect(day2, {'NOUN', 'ablt'})])

In [33]:
make_sec_sentence('ср', 'пн', 3)

'Скорость ветра изменится на 3 единицы в среду по сравнению с понедельником'

In [34]:
# Уровень осадков повысится / понизится на X единиц за Y дней.
def make_third_sentance(diff, days_count):
    return ' '.join(['Уровень осадков', 'повысится' if diff > 0 else 'понизится', 'на',
                     str(abs(diff)), inflect('единица', {'NOUN', 'gent'}), 'за', str(days_count),
                     'день' if days_count == 1 else (inflect('день', {'NOUN', 'gent'})\
                         if days_count < 5 else\
                             make_agree_with_number(inflect('день', {'NOUN', 'gent', 'plur'}), days_count))])

In [35]:
make_third_sentance(0.5, 3)

'Уровень осадков повысится на 0.5 единицы за 3 дня'

In [36]:
df.head()

,18.02 (вс),19.02 (пн),20.02 (вт),21.02 (ср),22.02 (чт),23.02 (пт),24.02 (сб),25.02 (вс),26.02 (пн),27.02 (вт)
минимальная температура,-6,-6,-9,-14,-17,-16,-18,-20,-23,-23
максимальная температура,-3,-3,-6,-10,-11,-11,-12,-16,-17,-15
скорость ветра,4,6,8,8,7,5,8,9,9,9
уровень осадков,1.85,1.5,0.8,0.1,0.1,0.2,0,0,0,0


In [37]:
for day1, day2 in combinations(df.columns, 2):
    print(make_first_sentence(day1[7:9], day2[7:9],
                        int(df[day2]['максимальная температура']) - int(df[day1]['максимальная температура'])))
    print(make_sec_sentence(day1[7:9], day2[7:9],
                            int(df[day2]['скорость ветра']) - int(df[day1]['скорость ветра'])))
    print(make_third_sentance(float(df[day2]['уровень осадков']) - float(df[day1]['уровень осадков']),
                              int(day2[:2]) - int(day1[:2])))
    print('-'*81)

В понедельник в Москве похолодает на 0 градусов по сравнению с воскресеньем
Скорость ветра изменится на 2 единицы в воскресенье по сравнению с понедельником
Уровень осадков понизится на 0.3500000000000001 единицы за 1 день
---------------------------------------------------------------------------------
Во вторник в Москве похолодает на 3 градуса по сравнению с воскресеньем
Скорость ветра изменится на 4 единицы в воскресенье по сравнению cо вторником
Уровень осадков понизится на 1.05 единицы за 2 дня
---------------------------------------------------------------------------------
В среду в Москве похолодает на 7 градусов по сравнению с воскресеньем
Скорость ветра изменится на 4 единицы в воскресенье по сравнению cо средой
Уровень осадков понизится на 1.75 единицы за 3 дня
---------------------------------------------------------------------------------
В четверг в Москве похолодает на 8 градусов по сравнению с воскресеньем
Скорость ветра изменится на 3 единицы в воскресенье по сравнен